In [14]:
import regex
import pandas as pd
import numpy as np

from Bio import Seq
from Bio import SeqIO

import cfg

def load_restriction_sites():

    re_seqs = []
    for record in SeqIO.parse(cfg.RESTRICTSITES_FN, 'fasta'):
        re_seqs.append(record.seq)
        re_seqs.append(record.seq.reverse_complement())

    return(set(re_seqs))

def rand_codon(aa, orig=None):
    '''
    select random codon
    based on hs genome freq
    '''
    codon_rows = AA_CODON_DT[AA_CODON_DT['aa'] == aa]
    if orig:
        codon_rows = codon_rows[codon_rows.codon != orig]
        if not len(codon_rows):
            #no other codons, can't use orig
            return(rand_codon(aa))
    return(np.random.choice(
            codon_rows.codon,
            p = codon_rows.freq/sum(codon_rows.freq)))

RE_SITES = load_restriction_sites()
RE_REGEX = regex.compile('('+'|'.join([str(re).replace('N','.') for re in RE_SITES])+')')

def load_hs_codon_freq():
    '''
    Generate aa to codon dictionary.
    '''
    aa_codon_dt = pd.read_csv(cfg.DATA_DIR+'hs_codon_freq.csv')
    aa_to_codon = dict(aa_codon_dt.loc[
            aa_codon_dt.groupby('aa')['freq'].idxmax()][
                    ['aa','codon']].to_dict('split')['data'])
    return(aa_codon_dt, aa_to_codon)

In [ ]:
def remove_re_sites(
        nt_seq, aa_seq,
        pre=cfg.OLIGO_CHECK_PRE,
        post=cfg.OLIGO_CHECK_POST,
        re_regex= RE_REGEX,
        v= False):

    assert(Bio.Seq.Seq(nt_seq).translate() == aa_seq)

    re_match = re_regex.search(pre+nt_seq+post, overlapped=True)
    replacements = 0
    codon_indices_replaced = []
    aa_replaced = []
    old_codons = []
    new_codons = []

    while re_match:

        if v:
            print('replacements:{}'.format(replacements))
            print(pre.lower() + nt_seq + post.lower())
            print('global match:'+str(re_match))

        if replacements > 100:
            raise Exception('too many codon replacement attempts') # something is up

        # get span of nucleotide indices
        nt_span = (
                max([0, re_match.span()[0]- len(pre)]),
                min([len(nt_seq), re_match.span()[1] - len(pre)]))

        # expand to corresponding codon indices
        aa_span = [i // 3 for i in nt_span]

        # convert codon indices back to nucleotides
        nt_rep_span = (aa_span[0]*3, aa_span[1]*3+3)

        if v: print('spans:'+str(nt_span)+str(aa_span)+str(nt_rep_span))

        # try replacing one codon at a time:
        for codon_i in random.sample(
                range(aa_span[0], aa_span[1]),
                aa_span[1]-aa_span[0]):

            old_codon = nt_seq[(codon_i*3):((codon_i*3)+3)]
            if v: print(codon_i)
            new_codon = rand_codon(aa_seq[codon_i], orig=old_codon)
            if v: print(new_codon)

            if v:
                print('changing codon {}:{}({}) to {}({})'.format(
                        codon_i,
                        old_codon,
                        aa_seq[codon_i],
                        new_codon,
                        Bio.Seq.Seq(new_codon).translate()))

            new_nt_seq = (
                    nt_seq[:(codon_i*3)] +
                    new_codon +
                    nt_seq[((codon_i*3)+3):])

            # print('new seq: '+pre+new_nt_seq+post)
            # test for same match as found originally
            local_re_match = re_regex.search((pre+new_nt_seq+post)[
                    re_match.span()[0]:re_match.span()[1]])
            if v: print('local match: '+str(local_re_match))

            # if no more match, we're done, accept new seq
            if not local_re_match:
                nt_seq = new_nt_seq
                codon_indices_replaced.append(codon_i)
                aa_replaced.append(aa_seq[codon_i])
                old_codons.append(old_codon)
                new_codons.append(new_codon)
                break

        replacements += 1
        #redo search to see if we still have restriction sites
        re_match = re_regex.search(pre+nt_seq+post)

    assert(Bio.Seq.Seq(nt_seq).translate() == aa_seq), '{} ne {}'.format(
            Bio.Seq.Seq(nt_seq).translate(),
            aa_seq)

    return(nt_seq, zip(
            codon_indices_replaced,
            aa_replaced,
            old_codons,
            new_codons))

In [11]:
load_restriction_sites()

{Seq('CGTCTC', SingleLetterAlphabet()),
 Seq('GAAGAC', SingleLetterAlphabet()),
 Seq('GAGACC', SingleLetterAlphabet()),
 Seq('GAGACG', SingleLetterAlphabet()),
 Seq('GGTCTC', SingleLetterAlphabet()),
 Seq('GTCTTC', SingleLetterAlphabet())}

In [12]:
RE_REGEX

regex.Regex('(GAGACG|GAAGAC|GTCTTC|GAGACC|CGTCTC|GGTCTC)', flags=regex.V0)